<a href="https://colab.research.google.com/github/KevinTheRainmaker/MLOps/blob/main/colabs/recommender/Recommender_Using_TF_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install -q tensorflow

In [36]:
! pip install -q tensorflow --upgrade --user

In [37]:
!pip install -q tensorflow-recommenders

In [38]:
!pip install -q --upgrade tensorflow-datasets

In [39]:
!pip install wandb -q

In [40]:
!wandb login

wandb: Currently logged in as: kevin_rain (use `wandb login --relogin` to force relogin)


In [41]:
import wandb

wandb.init(project='wandb_recommender',
           config={
               'layer_1_activation':"relu",
               'layer_1':256,
               'layer_2_activation':"relu",
               'layer_2':64,
               'learning_rate':0.01,
               'dropout_rate':0.2
           })

In [42]:
config = wandb.config

In [43]:
from wandb.keras import WandbCallback

In [44]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

print(tf.__version__)

2.6.0


In [45]:
ratings_raw = tfds.load("movielens/100k-ratings", split="train")
print(ratings_raw)

<PrefetchDataset shapes: {bucketized_user_age: (), movie_genres: (None,), movie_id: (), movie_title: (), raw_user_age: (), timestamp: (), user_gender: (), user_id: (), user_occupation_label: (), user_occupation_text: (), user_rating: (), user_zip_code: ()}, types: {bucketized_user_age: tf.float32, movie_genres: tf.int64, movie_id: tf.string, movie_title: tf.string, raw_user_age: tf.float32, timestamp: tf.int64, user_gender: tf.bool, user_id: tf.string, user_occupation_label: tf.int64, user_occupation_text: tf.string, user_rating: tf.float32, user_zip_code: tf.string}>


In [46]:
ratings = ratings_raw.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

In [47]:
shuffled = ratings.shuffle(100_000, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [48]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [49]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation='relu'),
      # tf.keras.layers.Dense(config.layer_1, activation=config.layer_1_activation),
      tf.keras.layers.Dense(64, activation='relu'),
      # tf.keras.layers.Dense(config.layer_2, activation=config.layer_2_activation),
      tf.keras.layers.Dropout(0.2),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [50]:
RankingModel()((["1"], ["One Flew Over the Cuckoo's Nest (1975)"]))

AttributeError: ignored

In [ ]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

# Compact

In [ ]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [ ]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.01))

In [ ]:
cached_train = train.shuffle(100_000).batch(4096).cache()
cached_test = test.batch(2048).cache()

In [ ]:
model.fit(cached_train, epochs=100)

Epoch 1/100
20/20 [==============================] - 5s 39ms/step - root_mean_squared_error: 2.7268 - loss: 7.0490 - regularization_loss: 0.0000e+00 - total_loss: 7.0490
Epoch 2/100
20/20 [==============================] - 1s 27ms/step - root_mean_squared_error: 1.1553 - loss: 1.3275 - regularization_loss: 0.0000e+00 - total_loss: 1.3275
Epoch 3/100
20/20 [==============================] - 1s 29ms/step - root_mean_squared_error: 1.1229 - loss: 1.2586 - regularization_loss: 0.0000e+00 - total_loss: 1.2586
Epoch 4/100
20/20 [==============================] - 1s 28ms/step - root_mean_squared_error: 1.1206 - loss: 1.2534 - regularization_loss: 0.0000e+00 - total_loss: 1.2534
Epoch 5/100
20/20 [==============================] - 1s 29ms/step - root_mean_squared_error: 1.1182 - loss: 1.2482 - regularization_loss: 0.0000e+00 - total_loss: 1.2482
Epoch 6/100
20/20 [==============================] - 1s 29ms/step - root_mean_squared_error: 1.1157 - loss: 1.2426 - regularization_loss: 0.0000e+00 -

In [ ]:
model.evaluate(cached_test, return_dict=True)

10/10 [==============================] - 3s 15ms/step - root_mean_squared_error: 0.9399 - loss: 0.8772 - regularization_loss: 0.0000e+00 - total_loss: 0.8772


{'loss': 0.8284850120544434,
 'regularization_loss': 0,
 'root_mean_squared_error': 0.9398870468139648,
 'total_loss': 0.8284850120544434}

In [ ]:
test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array(["42"]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
M*A*S*H (1970): [[4.076833]]
Dances with Wolves (1990): [[3.9437895]]
Speed (1994): [[3.7906384]]


# Serving

In [ ]:
tf.saved_model.save(model, "export")

INFO:tensorflow:Assets written to: export/assets


INFO:tensorflow:Assets written to: export/assets


In [ ]:
# Loading
loaded = tf.saved_model.load("export")

loaded({"user_id": np.array(["42"]), "movie_title": ["Speed (1994)"]}).numpy()

array([[3.7906384]], dtype=float32)